In [ ]:
from mxnet import autograd, np, npx, init
from mxnet.gluon import nn
from mxnet import gluon
from mxnet.gluon.data import ArrayDataset, DataLoader
from mxnet.image import imread
from mxnet.metric import Accuracy
import pandas as pd
import matplotlib.pyplot as plt
from mxnet.gluon.data.vision import transforms
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from IPython import display

npx.set_np()
ctx = npx.gpu()
sns.set_theme()

In [ ]:
df = pd.read_csv('../input/dog-breed-identification/labels.csv')
df.head()

## Define transforms for training and test
For the training set, we generating similar but distinct images by flipping and cropping.
For both sets we transforme image of shape (H, W, C) to (C, H, W), and normilize pixels.

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(331),
    transforms.RandomFlipLeftRight(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.255))
])

test_transform = transforms.Compose([
    transforms.Resize(350),
    transforms.CenterCrop(331),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.255))
])

class ImageLoader():
    def __init__(self, is_train=False):
        self._is_train = is_train
        
    def __call__(self, image_name, label=None):
        if self._is_train:
            folder = 'train'
            transform = train_transform
        else:
            folder = 'test'
            transform = test_transform
        
        path = f'../input/dog-breed-identification/{folder}/{image_name}.jpg'
        
        if label is None:
            return transform(imread(path))
        else:
            return transform(imread(path)), label

## Define DenseNet from scratch

In [ ]:
class ConvBlock(nn.Block):
    def __init__(self, num_channels, **kwargs):
        super().__init__(**kwargs)
        self.net = nn.Sequential()
        self.net.add(
            nn.BatchNorm(),
            nn.Activation('relu'),
            nn.Conv2D(num_channels, kernel_size=3, padding=1)
        )
    
    def forward(self, X):
        return self.net(X)


class DenseBlock(nn.Block):
    def __init__(self, num_convs, num_channels, **kwargs):
        super().__init__(**kwargs)
        self.net = nn.Sequential()
        
        for _ in range(num_convs):
            self.net.add(ConvBlock(num_channels))
        
    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            X = np.concatenate((X, Y), axis=1)
        
        return X


class TransitionBlock(nn.Block):
    def __init__(self, num_channels, **kwargs):
        super().__init__(**kwargs)
        self.net = nn.Sequential()
        self.net.add(
            nn.BatchNorm(),
            nn.Activation('relu'),
            nn.Conv2D(num_channels, kernel_size=1),
            nn.AvgPool2D(pool_size=2, strides=2)
        )
        
    def forward(self, X):
        return self.net(X)
        

class DenseNet(nn.Block):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.net = nn.Sequential()
        self.net.add(
            nn.Conv2D(64, kernel_size=7, strides=2, padding=3),
            nn.BatchNorm(),
            nn.Activation('relu'),
            nn.MaxPool2D(pool_size=3, strides=2, padding=1)
        )
        
        num_channels, growth_rate = 64, 32
        
        for i in range(4):
            self.net.add(DenseBlock(4, growth_rate))
            num_channels += 4 * growth_rate
            if i != 3:
                num_channels //= 2
                self.net.add(TransitionBlock(num_channels))
            
        self.net.add(
            nn.BatchNorm(),
            nn.Activation('relu'),
            nn.GlobalAvgPool2D(),
            nn.Dense(120)
        )
    
    def forward(self, X):
        return self.net(X)

## Training

In [ ]:
net = DenseNet()
net.initialize(init=init.Xavier(), ctx=ctx, force_reinit=True)

plt.ion()
figure, ax = plt.subplots(figsize=(8, 6))

lr, num_epochs, batch_size = 0.4, 300, 32
X = df['id']
y = df['breed']
count = len(y)
le = LabelEncoder()
le.fit(y.unique())
train_ds =  ArrayDataset(X.values, np.array(le.transform(y), ctx=ctx, dtype=np.float32))
train_dl = DataLoader(train_ds.transform(ImageLoader(is_train=True)), batch_size=batch_size)
loss = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd',
                        {'learning_rate': lr})

errors = []
acc = Accuracy()

for epoch in range(num_epochs):
    err = 0
    for X, y in train_dl:
        X, y = X.as_in_context(ctx), y.as_in_context(ctx)
    
        with autograd.record():
            y_hat = net(X)
            l = loss(y_hat, y)
        
        l.backward()
        err += l.sum()
        trainer.step(batch_size)

    errors.append(float(err) / count)
    ax.cla()
    ax.plot([i for i in range(epoch + 1)], errors, 'r')
    plt.title('Learning Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    plt.xlim(left=0)
    display.display(figure)
    display.clear_output(wait=True)   

In [ ]:
net.save_parameters('net.params')

## Make a prediction

In [ ]:
submission = pd.read_csv('../input/dog-breed-identification/sample_submission.csv')

for index, row in submission.iterrows():
    image = ImageLoader()(row.iloc[0]).as_in_context(ctx).reshape((1, 3, 331, 331))
    y_hat = net(image)
    y_hat = npx.softmax(y_hat).asnumpy().reshape((120,))
    submission.iloc[index, 1:] = y_hat

In [ ]:
submission.to_csv('submission.csv', index=False)
submission.head()